# Stokes Equation (2D)

## Falling Block (Constant Viscosity)

We begin with an instantaneous (time-independent) and isoviscous (constant viscosity) problem: the **falling block**. This setup is relatively simple to implement and serves as a useful benchmark problem for testing the existing code.  

### Problem

Unlike the full benchmark for this problem, we focus here only on the **isoviscous** case (in the benchmark, the sinking velocity and deformation of the block are studied as a function of the viscosity contrast [see here](../../examples/Benchmarks/)).  

We consider a square block of width (**W**), height (**T**), and density ($\rho_b$), embedded in a viscous medium of viscosity ($\eta_m$) and density ($\rho_m$). The computational domain is also square, with a length ($L = 500\ \textrm{km}$) and height ($H = 500\ \textrm{km}$). Throughout the domain, *free-slip* velocity boundary conditions are applied.  

<img src="../Figures/Exercise09_1.png" alt="drawing" width="450"/> <br>
**Figure 1.** Model setup

First, we load the necessary modules for visualizing results, solving the linear system, setting up the initial conditions, and solving the two-dimensional momentum conservation equation.  

A detailed description of how the model domain can be discretized into a numerical grid, as well as the discretization of **momentum conservation** and **mass conservation** under *free-slip* or *no-slip* boundary conditions, can be found [here](../../examples/StokesEquation/2D/README.md). Due to the scope of this exercise, we omit a detailed explanation.  

In [ ]:
using Plots
using ExtendableSparse
using GeoModBox.InitialCondition, GeoModBox.MomentumEquation.TwoD

Using the built-in function `IniPhase()`, we can generate the initial density distribution for our *block* problem. Depending on the setup, this function can create an initial distribution of certain quantities either on the numerical grid or on the markers.  

To use the function, we need the tuple `Ini`, in which the distribution is defined (in this case for the phase *p*).  

In [ ]:
# Define Initial Condition ========================================== #
# Density --- 
#   1) block
Ini         =   (p=:block,) 
# ------------------------------------------------------------------- #
# Plot Settings ===================================================== #
Pl  =   (
    qinc    =   5,
    qsc     =   100*(60*60*24*365.25)*1e2
)
# ------------------------------------------------------------------- #

Next, we define the geometry of our model domain.  

In [ ]:
# Geometry ========================================================== #
M       =   (
    xmin    =   0.0,
    xmax    =   500.0e3,    # [ m ]
    ymin    =   -500.0e3,   # [ m ]
    ymax    =   0.0,
)
# -------------------------------------------------------------------- #

Now we define our grid resolution $\left(nc_x = nc_y = 50\right)$ and construct the numerical grid.  

This requires specifying both the grid spacing, $\Delta{x}$ and $\Delta{y}$, as well as the coordinates of the different grid points (*vertices* and *centroids*). We first define the corresponding 1-D coordinate vectors, from which the 2-D coordinate grids are then generated.  

In [ ]:
# Grid =============================================================== #
NC      =   (
    x   =   50, 
    y   =   50,
)
NV      =   (
    x   =   NC.x + 1,
    y   =   NC.y + 1,
)
Δ       =   (
    x   =   (M.xmax - M.xmin)/NC.x,
    y   =   (M.ymax - M.ymin)/NC.y,
)
x       =   (
    c   =   LinRange(M.xmin+Δ.x/2,M.xmax-Δ.x/2,NC.x),
    ce  =   LinRange(M.xmin - Δ.x/2.0, M.xmax + Δ.x/2.0, NC.x+2),
    v   =   LinRange(M.xmin,M.xmax,NV.x),
)
y       =   (
    c   =   LinRange(M.ymin+Δ.y/2,M.ymax-Δ.y/2,NC.y),
    ce  =   LinRange(M.ymin - Δ.x/2.0, M.ymax + Δ.x/2.0, NC.y+2),
    v   =   LinRange(M.ymin,M.ymax,NV.y),
)
x1      =   (
    c2d     =   x.c .+ 0*y.c',
    v2d     =   x.v .+ 0*y.v', 
    vx2d    =   x.v .+ 0*y.ce',
    vy2d    =   x.ce .+ 0*y.v',
)
x   =   merge(x,x1)
y1      =   (
    c2d     =   0*x.c .+ y.c',
    v2d     =   0*x.v .+ y.v',
    vx2d    =   0*x.v .+ y.ce',
    vy2d    =   0*x.ce .+ y.v',
)
y   =   merge(y,y1)
# -------------------------------------------------------------------- #

Next, we need to define the physical parameters: gravitational acceleration $g$, viscosity $\eta_0$ ($10^{21}$ Pa·s), background density $\rho_0$ (3200 kg/m³), and block density $\rho_1$ (3300 kg/m³).  

The vector `phase` contains the phase index, which is used in the function `IniPhase()` to assign the corresponding phase to grid points or markers. The vector $\rho$ contains the density values associated with each phase.  

In [ ]:
# Physics ============================================================ #
g       =   9.81

η₀      =   1.0e21

ρ₀      =   3200.0          #   Background density
ρ₁      =   3300.0          #   Block density
ρ       =   [ρ₀,ρ₁] 

phase   =   [0,1]
# ------------------------------------------------------------------- #

Now we define the sizes of the required data fields.  

In [ ]:
# Allocation ======================================================== #
D   =   (
    vx      =   zeros(Float64,NV.x,NC.y+2),
    vy      =   zeros(Float64,NC.x+2,NV.y),
    Pt      =   zeros(Float64,NC...),
    p       =   zeros(Int64,NC...),
    p_ex    =   zeros(Int64,NC.x+2,NC.y+2),
    ρ       =   zeros(Float64,NC...),
    vxc     =   zeros(Float64,NC...),
    vyc     =   zeros(Float64,NC...),
    vc      =   zeros(Float64,NC...),
)
# ------------------------------------------------------------------- #

Now we can define the boundary conditions and the initial conditions.  

For the boundary conditions, we again use the tuple `VBC`, in which the type of boundary (*free slip* or *no slip*) is specified for each side (**E**, **W**, **S**, **N**).  
The assigned value `val` then determines the velocity value at the respective boundary.  

How do we initialize those conditions now in the script? 

In [ ]:
# Boundary Conditions =============================================== #
VBC     =   (
    type    =   (E=:freeslip,W=:freeslip,S=:freeslip,N=:freeslip),
    # type    =   (E=:noslip,W=:noslip,S=:noslip,N=:noslip),
    val     =   (E=zeros(NV.y),W=zeros(NV.y),S=zeros(NV.x),N=zeros(NV.x)),
)
# ------------------------------------------------------------------- #

To set up the initial conditions, we use the function `IniPhase`, which assigns the corresponding phase to each grid point.  
Each phase is then associated with its respective density value.  

In [ ]:
# Initial Condition ================================================= #
IniPhase!(Ini.p,D,M,x,y,NC;phase)
for i in eachindex(phase)
    D.ρ[D.p.==phase[i]] .= ρ[i]
end
# ------------------------------------------------------------------- #

Next, we need to define the parameters for the linear system of equations.  

The numbering of the equations for the *x-component* and *y-component* of the momentum conservation, as well as for the *mass conservation*, together with the discretization of the governing equations using the finite difference method, is described in detail [here](../../examples/StokesEquation/2D/README.md).  

In addition, the solution vector `χ` must be initialized. The size of both the coefficient matrix and the solution vector is determined by the maximum number of equations.  

In [ ]:
# System of Equations =============================================== #
# Numbering, without ghost nodes! ---
off    = [  NV.x*NC.y,                          # vx
            NV.x*NC.y + NC.x*NV.y,              # vy
            NV.x*NC.y + NC.x*NV.y + NC.x*NC.y]  # Pt

Num    =    (
    Vx  =   reshape(1:NV.x*NC.y, NV.x, NC.y), 
    Vy  =   reshape(off[1]+1:off[1]+NC.x*NV.y, NC.x, NV.y), 
    Pt  =   reshape(off[2]+1:off[2]+NC.x*NC.y,NC...),
)
χ       =   zeros(maximum(Num.Pt))
# ------------------------------------------------------------------- #

Using the function `Assemblyc()`, the coefficients of the coefficient matrix $\mathbf{K}$ can be assigned.  
This function associates the required coefficients with the corresponding equation in the system.  

In [ ]:
# Assemble Coefficients ============================================= #
K       =   Assemblyc(NC, NV, Δ, η₀, VBC, Num)
# ------------------------------------------------------------------- #

Next, we need to update the right-hand side (RHS) vector with the information from the initial and boundary conditions using the function `updaterhsc()`.  
This function assigns the appropriate values of the RHS to each equation in the linear system.  

In [ ]:
# Update RHS ======================================================== #
rhs     =   updaterhsc( NC, NV, Δ, η₀, D.ρ, -g, VBC, Num )
# ------------------------------------------------------------------- #

Now the problem can be solved by performing a right division.  

In [ ]:
# Solve System of Equations ========================================= #
χ       =   K \ rhs
# ------------------------------------------------------------------- #

Afterwards, the newly computed values for the horizontal and vertical velocity, as well as the pressure, must be assigned to the corresponding grid.  

In [ ]:
# Update Unknown Variables ========================================== #
D.vx[:,2:end-1]     .=  χ[Num.Vx]
D.vy[2:end-1,:]     .=  χ[Num.Vy]
D.Pt                .=  χ[Num.Pt]
# ------------------------------------------------------------------- #

For better visualization, we now compute the velocities at the *centroids*.  

In [ ]:
# Get the velocity on the centroids ---
for i = 1:NC.x
    for j = 1:NC.y
        D.vxc[i,j]  = (D.vx[i,j+1] + D.vx[i+1,j+1])/2
        D.vyc[i,j]  = (D.vy[i+1,j] + D.vy[i+1,j+1])/2
    end
end
@. D.vc        = sqrt(D.vxc^2 + D.vyc^2)

@show(minimum(D.vc))
@show(maximum(D.vc))


Finally, we visualize the density distribution and the velocity field.  

In [ ]:
p = heatmap(x.c./1e3,y.c./1e3,D.ρ',color=:inferno,
        xlabel="x[km]",ylabel="y[km]",colorbar=false,
        title="Density",
        aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3), 
        ylims=(M.ymin/1e3, M.ymax/1e3),
        layout=(2,2),subplot=1)
quiver!(p,x.c2d[1:Pl.qinc:end,1:Pl.qinc:end]./1e3,
        y.c2d[1:Pl.qinc:end,1:Pl.qinc:end]./1e3,
        quiver=(D.vx[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc,
        D.vyc[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc), 
        la=0.5,
        color="white",layout=(2,2),subplot=1)
heatmap!(p,x.c./1e3,y.c./1e3,D.vxc',
        xlabel="x[km]",ylabel="y[km]",colorbar=false,
        title="V_x",color=cgrad(:batlow),
        aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
        ylims=(M.ymin/1e3, M.ymax/1e3),
        layout=(2,2),subplot=3)
heatmap!(p,x.c./1e3,y.c./1e3,D.vyc',
        xlabel="x[km]",ylabel="y[km]",colorbar=false,
        title="V_y",color=cgrad(:batlow),
        aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
        ylims=(M.ymin/1e3, M.ymax/1e3),
        layout=(2,2),subplot=4)
heatmap!(p,x.c./1e3,y.c./1e3,D.Pt',
        xlabel="x[km]",ylabel="y[km]",colorbar=false,
        title="P_t",color=cgrad(:lipari),
        aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
        ylims=(M.ymin/1e3, M.ymax/1e3),
        layout=(2,2),subplot=2)
display(p)

savefig(p,string("./Results/09_FallingBlock_Instantan.png"))